# Crack 이미지로만 학습시키기 위해 txt 만들기

In [1]:
from glob import glob
import os

In [8]:
img_list = glob(os.path.join(os.getcwd(), 'dataset', 'images', '*.jpg'))
print(len(img_list))

20000


In [3]:
from sklearn.model_selection import train_test_split

train_image_list, validation_image_list = train_test_split(img_list, test_size=1/3, random_state=2000)
validation_image_list, test_image_list = train_test_split(validation_image_list, test_size=1/2, random_state=2000)

print(len(train_image_list), len(validation_image_list), len(test_image_list))

13333 3333 3334


In [4]:
test_image_list

['/home/piai/Desktop/crack/dataset/images/08500.jpg',
 '/home/piai/Desktop/crack/dataset/images/08705.jpg',
 '/home/piai/Desktop/crack/dataset/images/19102_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/12375_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/19429.jpg',
 '/home/piai/Desktop/crack/dataset/images/18853_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/18799_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/05375.jpg',
 '/home/piai/Desktop/crack/dataset/images/12856_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/14037_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/06228.jpg',
 '/home/piai/Desktop/crack/dataset/images/14609_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/09576.jpg',
 '/home/piai/Desktop/crack/dataset/images/04724.jpg',
 '/home/piai/Desktop/crack/dataset/images/05196.jpg',
 '/home/piai/Desktop/crack/dataset/images/11707_1.jpg',
 '/home/piai/Desktop/crack/dataset/images/02084.jpg',
 '/home/piai/Desktop/crack/dataset/images/02482.jpg',
 '/home/piai

train시 crack train, test, validation txt file 만들기

In [6]:
with open('./dataset/train.txt', mode='w') as f:
    f.write('\n'.join(train_image_list) + '\n')
    
with open('./dataset/test.txt', mode='w') as f:
    f.write('\n'.join(test_image_list) + '\n')
    
with open('./dataset/validation.txt', mode='w') as f:
    f.write('\n'.join(validation_image_list) + '\n')

# test_results/ 의 분석대상으로 사용할 테스트 이미지 3334장(1/6)
* detect.py 동작 시 넣을 예정

detect.py 시 사용할 test, noncrack test image folder 만들기

In [11]:
non_img_list = glob(os.path.join(os.getcwd(),'dataset','Negative','*.jpg'))

In [13]:
_, non_test_img_list = train_test_split(non_img_list, test_size=1/6, random_state=2000)

In [18]:
print(f'noncrack : {len(non_test_img_list)}, crack : {len(test_image_list)}')

noncrack : 3334, crack : 3334


In [31]:
from shutil import copyfile
crack_dest = os.path.join(os.getcwd(),'dataset','test_images','crack')
noncrack_dest = os.path.join(os.getcwd(),'dataset','test_images','non_crack')

In [25]:
# crack 이미지 복사
for source in test_image_list:
    filename = os.path.basename(source)
    dest = os.path.join(crack_dest, filename)
    copyfile(source, dest)

In [29]:
# non crack 이미지 복사
for source in non_test_img_list:
    filename = os.path.basename(source)
    dest = os.path.join(noncrack_dest, filename)
    copyfile(source, dest)

In [30]:
print(f'non crack : {len(os.listdir(noncrack_dest))}, crack : {len(os.listdir(crack_dest))}')

non crack : 3334, crack : 3334


In [38]:
len(os.listdir(os.path.join(os.getcwd(),'yolov5','runs','detect','crack')))

3335

# coco + crack
1. crack을 4:1:1로 분리
2. train crack + coco128
3. yaml에 txt 파일과 클래스 정보들을 기입한다(수동 or 코드 원하는대로)

### 결과
데이터의 불균형(학습시킬 crack 데이터(2만장)와 coco128(128장)으로 인해 detection결과 엉망

In [57]:
from glob import glob
import os

cracks = glob(os.path.join(os.getcwd(), 'dataset','pure_crack','images','*.jpg'))
coco128 = glob(os.path.join(os.getcwd(), 'dataset','coco128','images','train2017','*.jpg'))

In [58]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cracks, test_size=1/3, random_state=2000)
test, val = train_test_split(test, test_size=1/2, random_state=2000)

In [59]:
print(len(train), len(test), len(val))

13333 3333 3334


crack을 4 : 1 : 1로 나누고, train에 coco128를 붙인다

In [60]:
train = train + coco128

txt에 train, test, val 이미지 경로를 쓴다.

In [66]:
with open('./dataset/pure_crack/split_info/withcoco/train.txt', mode='w') as f:
    f.write('\n'.join(train) + '\n')
    
with open('./dataset/pure_crack/split_info/withcoco/test.txt', mode='w') as f:
    f.write('\n'.join(test) + '\n')
    
with open('./dataset/pure_crack/split_info/withcoco/validation.txt', mode='w') as f:
    f.write('\n'.join(val) + '\n')

# coco + selected cracks
* /dataset/SelectedCracks 에 선별된 crack 이미지와 label을 저장했다.
* class 번호는 80번으로, coco와 섞이지 않도록 했다.

### train : val : test = 4 : 1 : 1
* mAP@0.5=0.18403
* mAP_0.5:0.95=0.09834
* runs/trian/selected-result-66/labels.jpg를 참고하면 특정 클래스의 bbox가 많다.
* dataset/shittt 의 테스트이미지로 inference 한 결과도 딱히 우수하지 않았는데, 아래 이유인 것 같다
1. 데이터가 균질하지 않고
2. 데이터 수가 적음

* SDNET의 crack은 거의 못잡았다.(하나만 희미하게 잡음)

### train : val : test = 95 : 2.5 : 2.5
* mAP@0.5=0.4545
* mAP_0.5:0.95=0.2828
* 4:1:1의 비율로 분리 시 학습 데이터가 부족하므로 train에 대부분(95%)의 데이터를 집중시켜 학습시켰다.
* 지표는 위와 같이 4:1:1보다 좋았지만, 실제 inference해본 결과는 더 나빴다.
* SDNET의 crack들은 하나도 잡지 못했다

In [19]:
from glob import glob
import os

cracks = glob(os.path.join(os.getcwd(), 'dataset','SelectedCracks','images','*.jpg'))
coco128 = glob(os.path.join(os.getcwd(), 'dataset','coco128','images','train2017','*.jpg'))

In [20]:
datas = cracks + coco128

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(datas, test_size=1/3, random_state=2000)
test, validation = train_test_split(test, test_size=1/2, random_state=2000)

In [22]:
print(len(train), len(validation), len(test))

128 33 32


In [23]:
with open('./dataset/SelectedCracks/train.txt', mode='w') as f:
    f.write('\n'.join(train) + '\n')
    
with open('./dataset/SelectedCracks/validation.txt', mode='w') as f:
    f.write('\n'.join(validation) + '\n')
    
with open('./dataset/SelectedCracks/test.txt', mode='w') as f:
    f.write('\n'.join(test) + '\n')